In [ ]:
#use a ratio ofthe cbg's income level to the highest income level, to find how many in that cbg amy use bus - as a percentage of the total number of ODs

In [2]:
import pandas as pd
from gtfs_functions import Feed
import numpy as np

gtfs_path = '/home/rishav/Programs/routing/data/CARTA_GTFS2024'
# Load data
income_df = pd.read_csv('/home/rishav/Programs/routing/data/income_tn_all.csv', index_col=0) 
income_df['estimate'] = income_df['estimate'].astype(float)

od_df = pd.read_csv('/home/rishav/Programs/routing/outputs/lodes_2021-01-05_transit_drive2.csv', index_col=0)

od_df = od_df[od_df['transit_time'] > od_df['drive_time']]
od_df = od_df[od_df['walk_time'] > od_df['drive_time']]
od_df = od_df[od_df['walk_time'] >= od_df['transit_time']]

# Merge OD data with income data on CBG
data = pd.merge(od_df, income_df, left_on='h_geocode', right_on='GEOID', how='left')
data = data.dropna(subset='estimate')

In [3]:
max_income = income_df['estimate'].max()
data['income_ratio'] = data['estimate'] / max_income

In [4]:
# Normalize travel times
data['normalized_drive_time'] = data['drive_time'] / data['drive_time'].max()
data['normalized_transit_time'] = data['transit_time'] / data['transit_time'].max()

# Assign high penalty to missing bus times
high_penalty = 10  # Arbitrary high value; adjust as needed
data['normalized_transit_time'].fillna(high_penalty, inplace=True)


In [5]:
# Weights (tune these based on your specific needs or model calibration)
income_weight = 0.5
transit_time_weight = -0.5  # Negative because shorter times should increase bus usage
drive_time_weight = 0.5

# Calculate estimated bus usage probability
data['bus_usage_probability'] = (
    income_weight * data['income_ratio'] +
    transit_time_weight * data['normalized_transit_time'] +
    drive_time_weight * data['normalized_drive_time']
)

# Normalize probability to [0, 1] range
data['bus_usage_probability'] = (data['bus_usage_probability'] - data['bus_usage_probability'].min()) / (
        data['bus_usage_probability'].max() - data['bus_usage_probability'].min())


In [6]:
od_count_per_cbg = od_df.groupby('h_geocode').size().reset_index(name='total_od_count')
data = pd.merge(data, od_count_per_cbg, on='h_geocode', how='left')
data['estimated_bus_users'] = data['bus_usage_probability'] * data['total_od_count']


In [7]:
trips_df = pd.read_csv(f'{gtfs_path}/trips.txt')
stop_times_df = pd.read_csv(f'{gtfs_path}/stop_times.txt')
stop_times_df = stop_times_df[stop_times_df['departure_time']<'24:00:00']


# Convert arrival and departure times to a proper time format if necessary
stop_times_df['arrival_time'] = pd.to_datetime(stop_times_df['arrival_time'])
stop_times_df['departure_time'] = pd.to_datetime(stop_times_df['departure_time'])

# Assume you are interested in a specific time window
start_time = pd.to_datetime('06:00:00')
end_time = pd.to_datetime('10:00:00')

# Filter stop_times for trips that are operating within the time window
relevant_trips = stop_times_df[(stop_times_df['arrival_time'] >= start_time) & (stop_times_df['departure_time'] <= end_time)]

# Count the number of unique trips operating during this time
num_buses = relevant_trips['trip_id'].nunique()


In [9]:
# Merge trips data with the relevant trips to get route information
route_buses = relevant_trips.merge(trips_df, on='trip_id', how='left')

# Count the number of unique trips (buses) for each route
num_buses_per_route = route_buses.groupby('route_id')['trip_id'].nunique().reset_index(name='num_buses')


In [10]:
# Merge the number of buses per route back into your main dataset
data = pd.merge(data, num_buses_per_route, left_on='route1', right_on='route_id', how='left')

# Fill NaN values for routes without bus service in the time window with 0
data['num_buses'].fillna(0, inplace=True)


In [11]:
data['bus_capacity'] = 50
data['max_bus_capacity'] = data['bus_capacity'] * data['num_buses']
data['adjusted_bus_users'] = np.floor(data[['estimated_bus_users', 'max_bus_capacity']].min(axis=1))


In [12]:
data['random_number'] = np.random.rand(len(data))  # Generates a random number between 0 and 1 for each row
data['choice'] = np.where(data['random_number'] < data['bus_usage_probability'], 'Bus', 'Car')


In [14]:
# Count the initial number of bus users
bus_user_counts = data.groupby(['route_id', 'choice']).size().unstack(fill_value=0)

# Iterate over routes
for route_id, row in bus_user_counts.iterrows():
    if row['Bus'] > data.loc[data['route_id'] == route_id, 'max_bus_capacity'].values[0]:
        # Too many bus users, need to adjust
        excess = row['Bus'] - data.loc[data['route_id'] == route_id, 'max_bus_capacity'].values[0]
        bus_users = data[(data['route_id'] == route_id) & (data['choice'] == 'Bus')]
        drop_indices = bus_users.sample(n=int(excess)).index  # Randomly select excess users to switch to 'Car'
        data.loc[drop_indices, 'choice'] = 'Car'


In [20]:
data

,h_geocode,w_geocode,total_jobs,origin_loc_lat,origin_loc_lon,dest_loc_lat,dest_loc_lon,pickup_time_0,pickup_time_0_secs,pickup_time_0_str,...,bus_usage_probability,total_od_count,estimated_bus_users,random_number,choice,route_id,num_buses,bus_capacity,max_bus_capacity,adjusted_bus_users
0,470650108004,470650031002,33.0,35.114327,-85.300395,35.049290,-85.308987,09:35:00,34500.0,09:35:00,...,0.390694,264,103.143270,0.056026,Car,16,26.0,50,1300.0,103.0
1,470650108004,470650031002,33.0,35.120375,-85.295811,35.055210,-85.309259,10:42:30,38550.0,10:42:30,...,0.360633,264,95.207176,0.056161,Car,16,26.0,50,1300.0,95.0
2,470650108004,470650031002,33.0,35.115259,-85.296939,35.046263,-85.309068,10:33:45,38025.0,10:33:45,...,0.365523,264,96.498144,0.295654,Car,16,26.0,50,1300.0,96.0
3,470650108004,470650031002,33.0,35.120563,-85.295680,35.055885,-85.306213,10:21:00,37260.0,10:21:00,...,0.394582,264,104.169665,0.095502,Car,16,26.0,50,1300.0,104.0
4,470650108004,470650031002,33.0,35.115248,-85.295172,35.049290,-85.308987,06:30:15,23415.0,06:30:15,...,0.384035,264,101.385273,0.324029,Car,16,26.0,50,1300.0,101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84637,470650114481,470650118004,2.0,35.063880,-85.144764,34.995671,-85.238870,07:44:30,27870.0,07:44:30,...,0.314030,682,214.168465,0.862632,Car,4,46.0,50,2300.0,214.0
84638,470650104312,470650113212,1.0,35.180613,-85.244999,35.004621,-85.149968,07:10:30,25830.0,07:10:30,...,0.370005,621,229.773294,0.950089,Car,16,26.0,50,1300.0,229.0
84639,470650031001,470650105016,1.0,35.042982,-85.306228,35.112568,-85.254257,07:44:00,27840.0,07:44:00,...,0.464764,120,55.771721,0.196285,Car,16,26.0,50,1300.0,55.0
84640,470650113234,470650112052,2.0,35.036745,-85.135912,35.107574,-85.052015,06:37:45,23865.0,06:37:45,...,0.343714,420,144.360048,0.914892,Car,NaN,0.0,50,0.0,0.0
